In [ ]:
import sqlite3
from langchain.llms import GPT4All
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 1. Extraer metadata de Chinook.db
def extraer_metadata_sqlite(db_path):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # Obtener tablas
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tablas = [row[0] for row in cur.fetchall()]

    metadata = []
    for tabla in tablas:
        # Columnas y tipos
        cur.execute(f"PRAGMA table_info({tabla});")
        columnas = cur.fetchall()  # (cid, name, type, notnull, dflt_value, pk)
        columnas_str = ", ".join([f"{col[1]} ({col[2]})" for col in columnas])
        metadata.append(f"Tabla {tabla}: {columnas_str}")

    cur.close()
    conn.close()
    return metadata

# 2. Crear índice vectorial con metadata
def crear_vectorstore(metadata_texts):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(metadata_texts, embeddings)
    return vectorstore

# 3. Configurar LLM local GPT4All
llm = GPT4All(model="ggml-gpt4all-j-v1.3-groovy.bin")  # Ajusta el path a tu modelo

# 4. Prompt template
template = """
Eres un asistente que convierte lenguaje natural a SQL para SQLite.
Metadata relevante:
{metadata}

Consulta:
{query}

Genera la consulta SQL.
"""

prompt = PromptTemplate(input_variables=["metadata", "query"], template=template)
chain = LLMChain(llm=llm, prompt=prompt)

# 5. Ejecutar SQL en SQLite
def ejecutar_sql_sqlite(db_path, sql_query):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    try:
        cur.execute(sql_query)
        # Intentar obtener resultados si es SELECT
        if sql_query.strip().lower().startswith("select"):
            result = cur.fetchall()
        else:
            conn.commit()
            result = "Consulta ejecutada correctamente."
    except Exception as e:
        result = f"Error al ejecutar SQL: {e}"
    cur.close()
    conn.close()
    return result

# 6. Función principal
def text_to_sql_chinook(db_path, query_usuario, vectorstore):
    # Buscar metadata relevante
    docs = vectorstore.similarity_search(query_usuario, k=3)
    metadata_relevante = "\n".join([doc.page_content for doc in docs])

    # Generar SQL
    sql = chain.run(metadata=metadata_relevante, query=query_usuario)
    print("SQL generado:\n", sql)

    # Ejecutar SQL
    resultado = ejecutar_sql_sqlite(db_path, sql)
    print("Resultado:\n", resultado)

    return resultado

# --- Uso ---
if __name__ == "__main__":
    db_path = "Chinook.db"  # Ruta a tu base Chinook.db

    # Extraer metadata y crear índice (solo una vez)
    metadata_texts = extraer_metadata_sqlite(db_path)
    vectorstore = crear_vectorstore(metadata_texts)

    # Consulta de ejemplo
    consulta = "Muéstrame los nombres y correos de los clientes que han comprado música en 2010"
    respuesta = text_to_sql_chinook(db_path, consulta, vectorstore)
    print("\nRespuesta final:\n", respuesta)
